In [ ]:
import pandas as pd
import numpy as np
import json
import re

In [ ]:
#change 1,2
with open("2018-02-20_focus/2018-02-20_focus_group.json") as focus_group:
    focus = json.load(focus_group)

In [ ]:
with open("krx_code.json") as krx:
    krx_code = json.load(krx)

In [ ]:
import bs4
import urllib3

def scrape_price_history(code, time):
    '''
    code: string
    time: string, e.g. 201802200910
    '''

    target = "http://finance.naver.com/item/sise_time.nhn?code=" + code + "&thistime=" + time + "01&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    data_list = soup.find_all("tr")[2].find_all("td",class_="num")
    data_dic = {}
    data_dic["price"] = data_list[0].text
    image = data_list[1].find("img")
    if image != None:
        if image["alt"] == "상승":
            data_dic["price_dif"] = data_list[1].text.strip("\n\t")
        else:
            data_dic["price_dif"] = "-" + data_list[1].text.strip("\n\t")
    else:
        data_dic["price_dif"] = 0
    data_dic["sell"] = data_list[2].text
    data_dic["buy"] = data_list[3].text
    data_dic["volume"] = data_list[4].text
    data_dic["variation"] = data_list[5].text
    
    return data_dic

In [ ]:
time = []

#change 3,4
prefix = "2018-02-20_focus/discussion_2018-02-20"
for hour in range(9,16):
    for minute in range(0, 6):
        time.append(prefix + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ ":" + str(minute) + "0.json")

In [ ]:
time

In [ ]:
del time[41]
del time[40]

In [ ]:
df = pd.DataFrame(columns=["code", "name", "time", "price", "price_dif",\
                                        "sell", "buy", "volume", "variation"])

In [ ]:
for t in time:
    with open(t) as f:
        discussion = json.load(f)
    for d in discussion:
        row = pd.DataFrame(columns=["code", "name", "time", "price", "price_dif",\
                                    "sell", "buy", "volume", "variation"],\
                       data = [[krx_code[d["name"]], d["name"], d["time"], np.nan, np.nan,\
                               np.nan, np.nan, np.nan, np.nan]])
        df = df.append(row)

In [ ]:
df = df.reset_index()

In [ ]:
len(df)

In [ ]:
for idx, row in df.iterrows():
    timestamp = row["time"]
    t = re.sub('[ :-]', '', timestamp)

    d = scrape_price_history(row["code"], t)

    df.set_value(idx, "price", d["price"])
    df.set_value(idx, "price_dif", d["price_dif"])
    df.set_value(idx, "sell", d["sell"])
    df.set_value(idx, "buy", d["buy"])
    df.set_value(idx, "volume", d["volume"])
    df.set_value(idx, "variation", d["variation"])
    
    print(idx)

In [ ]:
rv = []
rv.append(df.columns.tolist())
for row in df.iterrows():
    rv.append(row[1].tolist())

In [ ]:
'''
import json
json = json.dumps(rv, ensure_ascii=False)
#change 5
f = open("2018-02-20_price_no_constraint.json","w")
f.write(json)
f.close()
'''